In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller
import math

In [2]:
csvs = ['nasdaq_screener_1685729611962.csv','nasdaq_screener_1685729316139.csv', 'nasdaq_screener_1685729644309.csv',
'nasdaq_screener_1685729362138.csv', 'nasdaq_screener_1685729666989.csv', 'nasdaq_screener_1685729424243.csv', 'nasdaq_screener_1685729708105.csv',
'nasdaq_screener_1685729454561.csv', 'nasdaq_screener_1685729730816.csv',
'nasdaq_screener_1685729483889.csv',
'nasdaq_screener_1685729579410.csv']

sector_to_symbol_dict = {} #maps sector to list of companies in that sector 

for csv in csvs:
    df = pd.read_csv(csv)
    for idx, row in df.iterrows():
        sector = row['Sector']
        sym = row['Symbol']
        
        if sector in sector_to_symbol_dict.keys():
            sector_to_symbol_dict[sector].append(sym)
        else:
            sector_to_symbol_dict[sector] = [sym]

In [3]:
sector_to_etf_dict = {} #maps sector to representative ETF
sector_to_etf_dict['Technology'] = 'XLK'
sector_to_etf_dict['Healthcare'] = 'XLV'
sector_to_etf_dict['Energy'] = 'XLE'
sector_to_etf_dict['Financials'] = 'XLF'
sector_to_etf_dict['Consumer Staples'] = 'XLP'
sector_to_etf_dict['Utilities'] = 'XLU'
sector_to_etf_dict['Consumer Discretionary'] = 'XLY'
sector_to_etf_dict['Basic Materials'] = 'XLB'
sector_to_etf_dict['Industrials'] = 'VIS'
sector_to_etf_dict['Telecommunications'] = 'VOX'
sector_to_etf_dict['Real Estate'] = 'XLRE'

In [4]:
sectors = list(sector_to_etf_dict.keys())
sectors

['Technology',
 'Healthcare',
 'Energy',
 'Financials',
 'Consumer Staples',
 'Utilities',
 'Consumer Discretionary',
 'Basic Materials',
 'Industrials',
 'Telecommunications',
 'Real Estate']

# Tech

In [5]:
tech_etf = sector_to_etf_dict['Technology']
tech_etf_df = yf.Ticker(tech_etf).history(period='max')

index = tech_etf_df.index
months = []
years = []
days = []

for dt in index:
    months.append(int(dt.month))
    years.append(int(dt.year))
    days.append(int(dt.day))

tech_etf_df['Month'] = pd.Series(months, index = index, dtype='int')
tech_etf_df['Year'] = pd.Series(years, index = index, dtype='int')
tech_etf_df['Day'] = pd.Series(days, index = index, dtype='int')

tech_etf_df.reset_index(drop=True, inplace=True)

In [6]:
tech_equities = sector_to_symbol_dict['Technology']
tech_eq_dfs = {}

for s in tech_equities:
    try:
        if s[len(s)-1] == 'W':
            continue
        dt = yf.Ticker(s).history(period='max').index[0]
        if pd.Timestamp(year=dt.year, month=dt.month, day=dt.day) > pd.Timestamp(year=int(tech_etf_df.iloc[0]['Year']), month = int(tech_etf_df.iloc[0]['Month']), day = int(tech_etf_df.iloc[0]['Day'])):
            df = yf.Ticker(s).history(period='max')
        else:
            if tech_etf_df.iloc[0]['Day'] < 10:
                df = yf.Ticker(s).history(start=str(int(tech_etf_df.iloc[0]['Year']))+'-'+str(int(tech_etf_df.iloc[0]['Month']))+'-0'+str(int(tech_etf_df.iloc[0]['Day'])))
            else:
                df = yf.Ticker(s).history(start=str(int(tech_etf_df.iloc[0]['Year']))+'-'+str(int(tech_etf_df.iloc[0]['Month']))+'-'+str(int(tech_etf_df.iloc[0]['Day'])))

        index = df.index
        months = []
        years = []
        days = []

        for dt in index:
            months.append(dt.month)
            years.append(dt.year)
            days.append(dt.day)

        df['Year'] = pd.Series(years, index = index)
        df['Month'] = pd.Series(months, index = index)
        df['Day'] = pd.Series(days, index = index)

        df.reset_index(drop=True, inplace=True)

        df = df.merge(right=tech_etf_df,how='inner',on=['Year','Month','Day'],suffixes=[None,' ETF'])
        tech_eq_dfs[s] = df
    except:
        continue 

LIFWZ: 1d data not available for startTime=-2208994789 and endTime=1686070495. Only 100 years worth of day granularity data are allowed to be fetched per request.


# Healthcare

In [7]:
hc_etf = sector_to_etf_dict['Healthcare']
hc_etf_df = yf.Ticker(hc_etf).history(period='max')

index = hc_etf_df.index
months = []
years = []
days = []

for dt in index:
    months.append(int(dt.month))
    years.append(int(dt.year))
    days.append(int(dt.day))

hc_etf_df['Month'] = pd.Series(months, index = index, dtype='int')
hc_etf_df['Year'] = pd.Series(years, index = index, dtype='int')
hc_etf_df['Day'] = pd.Series(days, index = index, dtype='int')

hc_etf_df.reset_index(drop=True, inplace=True)

In [8]:
hc_equities = sector_to_symbol_dict['Health Care']
hc_eq_dfs = {}

for idx in range(len(hc_equities)):
    s = hc_equities[idx]
    if idx % 100 == 0:
        print(idx)
    try:
        if len(s) >= 5:
            continue
        dt = yf.Ticker(s).history(period='max').index[0]
        if pd.Timestamp(year=dt.year, month=dt.month, day=dt.day) > pd.Timestamp(year=int(hc_etf_df.iloc[0]['Year']), month = int(hc_etf_df.iloc[0]['Month']), day = int(hc_etf_df.iloc[0]['Day'])):
            df = yf.Ticker(s).history(period='max')
        else:
            if hc_etf_df.iloc[0]['Day'] < 10:
                df = yf.Ticker(s).history(start=str(int(hc_etf_df.iloc[0]['Year']))+'-'+str(int(hc_etf_df.iloc[0]['Month']))+'-0'+str(int(hc_etf_df.iloc[0]['Day'])))
            else:
                df = yf.Ticker(s).history(start=str(int(hc_etf_df.iloc[0]['Year']))+'-'+str(int(hc_etf_df.iloc[0]['Month']))+'-'+str(int(hc_etf_df.iloc[0]['Day'])))

        index = df.index
        months = []
        years = []
        days = []

        for dt in index:
            months.append(dt.month)
            years.append(dt.year)
            days.append(dt.day)

        df['Year'] = pd.Series(years, index = index)
        df['Month'] = pd.Series(months, index = index)
        df['Day'] = pd.Series(days, index = index)

        df.reset_index(drop=True, inplace=True)

        df = df.merge(right=hc_etf_df,how='inner',on=['Year','Month','Day'],suffixes=[None,' ETF'])
        hc_eq_dfs[s] = df
    except:
        continue
    
    

0
100
200
300
400
500
600
700
800
900
1000


# Energy

In [9]:
energy_etf = sector_to_etf_dict['Energy']
energy_etf_df = yf.Ticker(energy_etf).history(period='max')

index = energy_etf_df.index
months = []
years = []
days = []

for dt in index:
    months.append(int(dt.month))
    years.append(int(dt.year))
    days.append(int(dt.day))

energy_etf_df['Month'] = pd.Series(months, index = index, dtype='int')
energy_etf_df['Year'] = pd.Series(years, index = index, dtype='int')
energy_etf_df['Day'] = pd.Series(days, index = index, dtype='int')

energy_etf_df.reset_index(drop=True, inplace=True)

In [10]:
energy_equities = sector_to_symbol_dict['Energy']
energy_eq_dfs = {}

for idx in range(len(energy_equities)):
    if idx % 50 == 0:
        print(idx)
    s = energy_equities[idx]
    try:
        if len(s) >= 5:
            continue
        dt = yf.Ticker(s).history(period='max').index[0]
        if pd.Timestamp(year=dt.year, month=dt.month, day=dt.day) > pd.Timestamp(year=int(energy_etf_df.iloc[0]['Year']), month = int(energy_etf_df.iloc[0]['Month']), day = int(energy_etf_df.iloc[0]['Day'])):
            df = yf.Ticker(s).history(period='max')
        else:
            if energy_etf_df.iloc[0]['Day'] < 10:
                df = yf.Ticker(s).history(start=str(int(energy_etf_df.iloc[0]['Year']))+'-'+str(int(energy_etf_df.iloc[0]['Month']))+'-0'+str(int(energy_etf_df.iloc[0]['Day'])))
            else:
                df = yf.Ticker(s).history(start=str(int(energy_etf_df.iloc[0]['Year']))+'-'+str(int(energy_etf_df.iloc[0]['Month']))+'-'+str(int(energy_etf_df.iloc[0]['Day'])))

        index = df.index
        months = []
        years = []
        days = []

        for dt in index:
            months.append(dt.month)
            years.append(dt.year)
            days.append(dt.day)

        df['Year'] = pd.Series(years, index = index)
        df['Month'] = pd.Series(months, index = index)
        df['Day'] = pd.Series(days, index = index)

        df.reset_index(drop=True, inplace=True)

        df = df.merge(right=energy_etf_df,how='inner',on=['Year','Month','Day'],suffixes=[None,' ETF'])
        energy_eq_dfs[s] = df
    except:
        continue
    
    

0
50
100
150


# Financials

In [11]:
fin_etf = sector_to_etf_dict['Financials']
fin_etf_df = yf.Ticker(fin_etf).history(period='max')

index = fin_etf_df.index
months = []
years = []
days = []

for dt in index:
    months.append(int(dt.month))
    years.append(int(dt.year))
    days.append(int(dt.day))

fin_etf_df['Month'] = pd.Series(months, index = index, dtype='int')
fin_etf_df['Year'] = pd.Series(years, index = index, dtype='int')
fin_etf_df['Day'] = pd.Series(days, index = index, dtype='int')

fin_etf_df.reset_index(drop=True, inplace=True)

In [12]:
fin_equities = sector_to_symbol_dict['Finance']
fin_eq_dfs = {}

for idx in range(len(fin_equities)):
    if idx % 100 == 0:
        print(idx)
    s = fin_equities[idx]
    try:
        if len(s) >= 5:
            continue
        dt = yf.Ticker(s).history(period='max').index[0]
        if pd.Timestamp(year=dt.year, month=dt.month, day=dt.day) > pd.Timestamp(year=int(fin_etf_df.iloc[0]['Year']), month = int(fin_etf_df.iloc[0]['Month']), day = int(fin_etf_df.iloc[0]['Day'])):
            df = yf.Ticker(s).history(period='max')
        else:
            if fin_etf_df.iloc[0]['Day'] < 10:
                df = yf.Ticker(s).history(start=str(int(fin_etf_df.iloc[0]['Year']))+'-'+str(int(fin_etf_df.iloc[0]['Month']))+'-0'+str(int(fin_etf_df.iloc[0]['Day'])))
            else:
                df = yf.Ticker(s).history(start=str(int(fin_etf_df.iloc[0]['Year']))+'-'+str(int(fin_etf_df.iloc[0]['Month']))+'-'+str(int(fin_etf_df.iloc[0]['Day'])))

        index = df.index
        months = []
        years = []
        days = []

        for dt in index:
            months.append(dt.month)
            years.append(dt.year)
            days.append(dt.day)

        df['Year'] = pd.Series(years, index = index)
        df['Month'] = pd.Series(months, index = index)
        df['Day'] = pd.Series(days, index = index)

        df.reset_index(drop=True, inplace=True)

        df = df.merge(right=fin_etf_df,how='inner',on=['Year','Month','Day'],suffixes=[None,' ETF'])
        fin_eq_dfs[s] = df
    except:
        continue
    

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
SFB: 1d data not available for startTime=-2208994789 and endTime=1686071634. Only 100 years worth of day granularity data are allowed to be fetched per request.
1400
1500


# Consumer Staples

In [13]:
cs_etf = sector_to_etf_dict['Consumer Staples']
cs_etf_df = yf.Ticker(cs_etf).history(period='max')

index = cs_etf_df.index
months = []
years = []
days = []

for dt in index:
    months.append(int(dt.month))
    years.append(int(dt.year))
    days.append(int(dt.day))

cs_etf_df['Month'] = pd.Series(months, index = index, dtype='int')
cs_etf_df['Year'] = pd.Series(years, index = index, dtype='int')
cs_etf_df['Day'] = pd.Series(days, index = index, dtype='int')

cs_etf_df.reset_index(drop=True, inplace=True)

In [14]:
cs_equities = sector_to_symbol_dict['Consumer Staples']
cs_eq_dfs = {}

for idx in range(len(cs_equities)):
    if idx % 10 == 0:
        print(idx)
    s = cs_equities[idx]
    try:
        if len(s) >= 5:
            continue
        dt = yf.Ticker(s).history(period='max').index[0]
        if pd.Timestamp(year=dt.year, month=dt.month, day=dt.day) > pd.Timestamp(year=int(cs_etf_df.iloc[0]['Year']), month = int(cs_etf_df.iloc[0]['Month']), day = int(cs_etf_df.iloc[0]['Day'])):
            df = yf.Ticker(s).history(period='max')
        else:
            if cs_etf_df.iloc[0]['Day'] < 10:
                df = yf.Ticker(s).history(start=str(int(cs_etf_df.iloc[0]['Year']))+'-'+str(int(cs_etf_df.iloc[0]['Month']))+'-0'+str(int(cs_etf_df.iloc[0]['Day'])))
            else:
                df = yf.Ticker(s).history(start=str(int(cs_etf_df.iloc[0]['Year']))+'-'+str(int(cs_etf_df.iloc[0]['Month']))+'-'+str(int(cs_etf_df.iloc[0]['Day'])))

        index = df.index
        months = []
        years = []
        days = []

        for dt in index:
            months.append(dt.month)
            years.append(dt.year)
            days.append(dt.day)

        df['Year'] = pd.Series(years, index = index)
        df['Month'] = pd.Series(months, index = index)
        df['Day'] = pd.Series(days, index = index)

        df.reset_index(drop=True, inplace=True)

        df = df.merge(right=cs_etf_df,how='inner',on=['Year','Month','Day'],suffixes=[None,' ETF'])
        cs_eq_dfs[s] = df
    except:
        continue

0
10
20
30
40
50
60
70
80
90
100


# Utilities

In [15]:
util_etf = sector_to_etf_dict['Utilities']
util_etf_df = yf.Ticker(util_etf).history(period='max')

index = util_etf_df.index
months = []
years = []
days = []

for dt in index:
    months.append(int(dt.month))
    years.append(int(dt.year))
    days.append(int(dt.day))

util_etf_df['Month'] = pd.Series(months, index = index, dtype='int')
util_etf_df['Year'] = pd.Series(years, index = index, dtype='int')
util_etf_df['Day'] = pd.Series(days, index = index, dtype='int')

util_etf_df.reset_index(drop=True, inplace=True)

In [16]:
util_equities = sector_to_symbol_dict['Utilities']
util_eq_dfs = {}

for idx in range(len(util_equities)):
    if idx % 10 == 0:
        print(idx)
    s = util_equities[idx]
    try:
        if len(s) >= 5:
            continue
        dt = yf.Ticker(s).history(period='max').index[0]
        if pd.Timestamp(year=dt.year, month=dt.month, day=dt.day) > pd.Timestamp(year=int(util_etf_df.iloc[0]['Year']), month = int(util_etf_df.iloc[0]['Month']), day = int(util_etf_df.iloc[0]['Day'])):
            df = yf.Ticker(s).history(period='max')
        else:
            if util_etf_df.iloc[0]['Day'] < 10:
                df = yf.Ticker(s).history(start=str(int(util_etf_df.iloc[0]['Year']))+'-'+str(int(util_etf_df.iloc[0]['Month']))+'-0'+str(int(util_etf_df.iloc[0]['Day'])))
            else:
                df = yf.Ticker(s).history(start=str(int(util_etf_df.iloc[0]['Year']))+'-'+str(int(util_etf_df.iloc[0]['Month']))+'-'+str(int(util_etf_df.iloc[0]['Day'])))

        index = df.index
        months = []
        years = []
        days = []

        for dt in index:
            months.append(dt.month)
            years.append(dt.year)
            days.append(dt.day)

        df['Year'] = pd.Series(years, index = index)
        df['Month'] = pd.Series(months, index = index)
        df['Day'] = pd.Series(days, index = index)

        df.reset_index(drop=True, inplace=True)

        df = df.merge(right=util_etf_df,how='inner',on=['Year','Month','Day'],suffixes=[None,' ETF'])
        util_eq_dfs[s] = df
    except:
        continue
        

0
10
20
30
40
EAI: No data found for this date range, symbol may be delisted
50
60
70
80
90
100
110
120
130


# Consumer Discretionary

In [17]:
cd_etf = sector_to_etf_dict['Consumer Discretionary']
cd_etf_df = yf.Ticker(cd_etf).history(period='max')

index = cd_etf_df.index
months = []
years = []
days = []

for dt in index:
    months.append(int(dt.month))
    years.append(int(dt.year))
    days.append(int(dt.day))

cd_etf_df['Month'] = pd.Series(months, index = index, dtype='int')
cd_etf_df['Year'] = pd.Series(years, index = index, dtype='int')
cd_etf_df['Day'] = pd.Series(days, index = index, dtype='int')

cd_etf_df.reset_index(drop=True, inplace=True)

In [18]:
cd_equities = sector_to_symbol_dict['Consumer Discretionary']
cd_eq_dfs = {}

for idx in range(len(cd_equities)):
    if idx % 100 == 0:
        print(idx)
    s = cd_equities[idx]
    try:
        if len(s) >= 5:
            continue
        dt = yf.Ticker(s).history(period='max').index[0]
        if pd.Timestamp(year=dt.year, month=dt.month, day=dt.day) > pd.Timestamp(year=int(cd_etf_df.iloc[0]['Year']), month = int(cd_etf_df.iloc[0]['Month']), day = int(cd_etf_df.iloc[0]['Day'])):
            df = yf.Ticker(s).history(period='max')
        else:
            if cd_etf_df.iloc[0]['Day'] < 10:
                df = yf.Ticker(s).history(start=str(int(cd_etf_df.iloc[0]['Year']))+'-'+str(int(cd_etf_df.iloc[0]['Month']))+'-0'+str(int(cd_etf_df.iloc[0]['Day'])))
            else:
                df = yf.Ticker(s).history(start=str(int(cd_etf_df.iloc[0]['Year']))+'-'+str(int(cd_etf_df.iloc[0]['Month']))+'-'+str(int(cd_etf_df.iloc[0]['Day'])))

        index = df.index
        months = []
        years = []
        days = []

        for dt in index:
            months.append(dt.month)
            years.append(dt.year)
            days.append(dt.day)

        df['Year'] = pd.Series(years, index = index)
        df['Month'] = pd.Series(months, index = index)
        df['Day'] = pd.Series(days, index = index)

        df.reset_index(drop=True, inplace=True)

        df = df.merge(right=cd_etf_df,how='inner',on=['Year','Month','Day'],suffixes=[None,' ETF'])
        cd_eq_dfs[s] = df
    except:
        continue
    

0
100
CSGP: Error occurred while retrieving timeseries from Redis, keys: [RedisKey [key=CSGP, cluster=finance]]
200
300
400
500
600
700
800


# Basic Materials

In [19]:
bm_etf = sector_to_etf_dict['Basic Materials']
bm_etf_df = yf.Ticker(bm_etf).history(period='max')

index = bm_etf_df.index
months = []
years = []
days = []

for dt in index:
    months.append(int(dt.month))
    years.append(int(dt.year))
    days.append(int(dt.day))

bm_etf_df['Month'] = pd.Series(months, index = index, dtype='int')
bm_etf_df['Year'] = pd.Series(years, index = index, dtype='int')
bm_etf_df['Day'] = pd.Series(days, index = index, dtype='int')

bm_etf_df.reset_index(drop=True, inplace=True)

In [20]:
bm_equities = sector_to_symbol_dict['Basic Materials']
bm_eq_dfs = {}

for idx in range(len(bm_equities)):
    if idx % 100 == 0:
        print(idx)
    s = bm_equities[idx]
    try:
        if len(s) >= 5:
            continue
        dt = yf.Ticker(s).history(period='max').index[0]
        if pd.Timestamp(year=dt.year, month=dt.month, day=dt.day) > pd.Timestamp(year=int(bm_etf_df.iloc[0]['Year']), month = int(bm_etf_df.iloc[0]['Month']), day = int(bm_etf_df.iloc[0]['Day'])):
            df = yf.Ticker(s).history(period='max')
        else:
            if bm_etf_df.iloc[0]['Day'] < 10:
                df = yf.Ticker(s).history(start=str(int(bm_etf_df.iloc[0]['Year']))+'-'+str(int(bm_etf_df.iloc[0]['Month']))+'-0'+str(int(bm_etf_df.iloc[0]['Day'])))
            else:
                df = yf.Ticker(s).history(start=str(int(bm_etf_df.iloc[0]['Year']))+'-'+str(int(bm_etf_df.iloc[0]['Month']))+'-'+str(int(bm_etf_df.iloc[0]['Day'])))

        index = df.index
        months = []
        years = []
        days = []

        for dt in index:
            months.append(dt.month)
            years.append(dt.year)
            days.append(dt.day)

        df['Year'] = pd.Series(years, index = index)
        df['Month'] = pd.Series(months, index = index)
        df['Day'] = pd.Series(days, index = index)

        df.reset_index(drop=True, inplace=True)

        df = df.merge(right=bm_etf_df,how='inner',on=['Year','Month','Day'],suffixes=[None,' ETF'])
        bm_eq_dfs[s] = df
    except:
        continue

0


# Industrials

In [21]:
ind_etf = sector_to_etf_dict['Industrials']
ind_etf_df = yf.Ticker(ind_etf).history(period='max')

index = ind_etf_df.index
months = []
years = []
days = []

for dt in index:
    months.append(int(dt.month))
    years.append(int(dt.year))
    days.append(int(dt.day))

ind_etf_df['Month'] = pd.Series(months, index = index, dtype='int')
ind_etf_df['Year'] = pd.Series(years, index = index, dtype='int')
ind_etf_df['Day'] = pd.Series(days, index = index, dtype='int')

ind_etf_df.reset_index(drop=True, inplace=True)

In [22]:
ind_equities = sector_to_symbol_dict['Industrials']
ind_eq_dfs = {}

for idx in range(len(ind_equities)):
    if idx % 100 == 0:
        print(idx)
    s = ind_equities[idx]
    try:
        if len(s) >= 5:
            continue
        dt = yf.Ticker(s).history(period='max').index[0]
        if pd.Timestamp(year=dt.year, month=dt.month, day=dt.day) > pd.Timestamp(year=int(ind_etf_df.iloc[0]['Year']), month = int(ind_etf_df.iloc[0]['Month']), day = int(ind_etf_df.iloc[0]['Day'])):
            df = yf.Ticker(s).history(period='max')
        else:
            if ind_etf_df.iloc[0]['Day'] < 10:
                df = yf.Ticker(s).history(start=str(int(ind_etf_df.iloc[0]['Year']))+'-'+str(int(ind_etf_df.iloc[0]['Month']))+'-0'+str(int(ind_etf_df.iloc[0]['Day'])))
            else:
                df = yf.Ticker(s).history(start=str(int(ind_etf_df.iloc[0]['Year']))+'-'+str(int(ind_etf_df.iloc[0]['Month']))+'-'+str(int(ind_etf_df.iloc[0]['Day'])))

        index = df.index
        months = []
        years = []
        days = []

        for dt in index:
            months.append(dt.month)
            years.append(dt.year)
            days.append(dt.day)

        df['Year'] = pd.Series(years, index = index)
        df['Month'] = pd.Series(months, index = index)
        df['Day'] = pd.Series(days, index = index)

        df.reset_index(drop=True, inplace=True)

        df = df.merge(right=ind_etf_df,how='inner',on=['Year','Month','Day'],suffixes=[None,' ETF'])
        ind_eq_dfs[s] = df
    except:
        continue
    

0
100
200
300
400


# Telecommunications

In [23]:
tc_etf = sector_to_etf_dict['Telecommunications']
tc_etf_df = yf.Ticker(tc_etf).history(period='max')

index = tc_etf_df.index
months = []
years = []
days = []

for dt in index:
    months.append(int(dt.month))
    years.append(int(dt.year))
    days.append(int(dt.day))

tc_etf_df['Month'] = pd.Series(months, index = index, dtype='int')
tc_etf_df['Year'] = pd.Series(years, index = index, dtype='int')
tc_etf_df['Day'] = pd.Series(days, index = index, dtype='int')

tc_etf_df.reset_index(drop=True, inplace=True)

In [24]:
tc_equities = sector_to_symbol_dict['Telecommunications']
tc_eq_dfs = {}

for idx in range(len(tc_equities)):
    if idx % 100 == 0:
        print(idx)
    s = tc_equities[idx]
    try:
        if len(s) >= 5:
            continue
        dt = yf.Ticker(s).history(period='max').index[0]
        if pd.Timestamp(year=dt.year, month=dt.month, day=dt.day) > pd.Timestamp(year=int(tc_etf_df.iloc[0]['Year']), month = int(tc_etf_df.iloc[0]['Month']), day = int(tc_etf_df.iloc[0]['Day'])):
            df = yf.Ticker(s).history(period='max')
        else:
            if tc_etf_df.iloc[0]['Day'] < 10:
                df = yf.Ticker(s).history(start=str(int(tc_etf_df.iloc[0]['Year']))+'-'+str(int(tc_etf_df.iloc[0]['Month']))+'-0'+str(int(tc_etf_df.iloc[0]['Day'])))
            else:
                df = yf.Ticker(s).history(start=str(int(tc_etf_df.iloc[0]['Year']))+'-'+str(int(tc_etf_df.iloc[0]['Month']))+'-'+str(int(tc_etf_df.iloc[0]['Day'])))

        index = df.index
        months = []
        years = []
        days = []

        for dt in index:
            months.append(dt.month)
            years.append(dt.year)
            days.append(dt.day)

        df['Year'] = pd.Series(years, index = index)
        df['Month'] = pd.Series(months, index = index)
        df['Day'] = pd.Series(days, index = index)

        df.reset_index(drop=True, inplace=True)

        df = df.merge(right=tc_etf_df,how='inner',on=['Year','Month','Day'],suffixes=[None,' ETF'])
        tc_eq_dfs[s] = df
    except:
        continue
        

0


# Real Estate

In [25]:
re_etf = sector_to_etf_dict['Real Estate']
re_etf_df = yf.Ticker(re_etf).history(period='max')

index = re_etf_df.index
months = []
years = []
days = []

for dt in index:
    months.append(int(dt.month))
    years.append(int(dt.year))
    days.append(int(dt.day))

re_etf_df['Month'] = pd.Series(months, index = index, dtype='int')
re_etf_df['Year'] = pd.Series(years, index = index, dtype='int')
re_etf_df['Day'] = pd.Series(days, index = index, dtype='int')

re_etf_df.reset_index(drop=True, inplace=True)

In [26]:
re_equities = sector_to_symbol_dict['Real Estate']
re_eq_dfs = {}

for idx in range(len(re_equities)):
    if idx % 100 == 0:
        print(idx)
    s = re_equities[idx]
    try:
        if len(s) >= 5:
            continue
        dt = yf.Ticker(s).history(period='max').index[0]
        if pd.Timestamp(year=dt.year, month=dt.month, day=dt.day) > pd.Timestamp(year=int(re_etf_df.iloc[0]['Year']), month = int(re_etf_df.iloc[0]['Month']), day = int(re_etf_df.iloc[0]['Day'])):
            df = yf.Ticker(s).history(period='max')
        else:
            if re_etf_df.iloc[0]['Day'] < 10:
                df = yf.Ticker(s).history(start=str(int(re_etf_df.iloc[0]['Year']))+'-'+str(int(re_etf_df.iloc[0]['Month']))+'-0'+str(int(re_etf_df.iloc[0]['Day'])))
            else:
                df = yf.Ticker(s).history(start=str(int(re_etf_df.iloc[0]['Year']))+'-'+str(int(re_etf_df.iloc[0]['Month']))+'-'+str(int(re_etf_df.iloc[0]['Day'])))

        index = df.index
        months = []
        years = []
        days = []

        for dt in index:
            months.append(dt.month)
            years.append(dt.year)
            days.append(dt.day)

        df['Year'] = pd.Series(years, index = index)
        df['Month'] = pd.Series(months, index = index)
        df['Day'] = pd.Series(days, index = index)

        df.reset_index(drop=True, inplace=True)

        df = df.merge(right=re_etf_df,how='inner',on=['Year','Month','Day'],suffixes=[None,' ETF'])
        re_eq_dfs[s] = df
    except:
        continue
    
    

0
100
RCA: No data found for this date range, symbol may be delisted
200


for each of the 11 stock sectors, I have a dictionary mapping symbol to price dataframe with ETF prices included in the dataframe for each stock

choose a lookback period during which data is used to form pairs; choose a trading period, during which pairs and trading rules that were formed based on lookback period, such that pairs trading occurs. For each asset, etf pair, loop thru all rolling windows of lookback period months. For each asset, etf pair and each rolling window and trading period, check that prices exceed some correlation threshold and that the residuals are stationary. Then, construct trading rules and pairs trade and record performance. Hyperparameters are lookback period length, trading period length, correlation threshold, and p-value for cointegration test. 

In [28]:
lookback = 6 #6 months
trading_period = 3 #3 months
corr_thresh = 0.8
pval_max = 0.05

In [36]:
def outsample_pairs_trade(spread, so, lo, sc, lc, max_pos_period):
    #so: short open point
    #lo: long open point
    #sc: short close point
    #lc: long close point
    #max_pos_period: maximum number of days you want the position to be open
    
    prof = 0
    num_trades = 0
    
    short = False
    long_ = False
    open_idx = 0
    
    for idx in range(spread.shape[0]):
        
        if short:
            #currently have an open position where we are shorting the spread
            
            if spread[idx] <= sc or idx - open_idx >= max_pos_period:
                #close the position
                short = False
                prof += spread[open_idx] - spread[idx]
            elif idx == spread.shape[0] - 1:
                #last day of data so close position
                prof += spread[open_idx] - spread[idx]
        
        elif long_:
            #currently have an open position where we are long-ing the spread
            
            if spread[idx] >= lc or idx - open_idx >= max_pos_period:
                #close
                long_ = False
                prof += spread[idx] - spread[open_idx]
            elif idx == spread.shape[0] - 1:
                #last day of data so close position
                prof += spread[idx] - spread[open_idx]
        else:
            
            #determine if we will open a position
            if spread[idx] >= so and idx < spread.shape[0] - 1:
                short = True
                open_idx = idx
                num_trades+=1
            elif spread[idx] <= lo and idx < spread.shape[0] - 1:
                long_ = True
                open_idx = idx
                num_trades+=1
                
    
    return prof, num_trades
    

# Tech

In [33]:
tech_trade_params = {} #maps (tech equities, start of window year-month tuple) tuple to values that need to be known to pairs trade [alpha, beta, mean, sd]
tech_equities = tech_eq_dfs.keys()
idx = 0
for s in tech_equities:
    idx+=1
    if idx % 50 == 0:
        print(idx*100/len(tech_equities))
    df = tech_eq_dfs[s]
    gbo = df.groupby(['Year','Month'])
    gb_keys = []
    for key, frame in gbo:
        gb_keys.append(key)
    for y_m_key in gb_keys[:-8]:
        lookback_df_list = [] 
        first_year = y_m_key[0]
        first_month = y_m_key[1]
        for i in range(6):
            cur_month = first_month + i
            if cur_month < 13:
                cur_year = first_year
            else:
                cur_month-=12
                cur_year = first_year+1
            cur_gp = gbo.get_group((cur_year,cur_month))
            lookback_df_list.append(cur_gp)
        lookback_df = lookback_df_list[0]
        for i in range(1,len(lookback_df_list)):
            lookback_df = pd.concat([lookback_df,lookback_df_list[i]],ignore_index=True)
        try:
            ols = sm.OLS(np.array(lookback_df['Close']),sm.add_constant(np.array(lookback_df['Close ETF']).reshape((-1,1))))
            res = ols.fit()
            alpha = res.params[0] #y-int
            beta = res.params[1]
            residuals = np.subtract(np.array(lookback_df['Close']),ols.predict(res.params)) + np.ones(lookback_df.shape[0])*res.params[0]
            cur_pval = adfuller(residuals)[1]
            corr_coef = np.corrcoef(np.hstack((np.array(lookback_df['Close']).reshape(-1),np.array(lookback_df['Close ETF']).reshape(-1))))
            if cur_pval <= pval_max and corr_coef >= corr_thresh:
                #can trade spread
                mean = np.mean(residuals)
                sd = np.std(residuals)
                tech_trade_params[(s,(y_m_key))] = [alpha,beta,mean,sd]
        except:
            continue
        

9.523809523809524
19.047619047619047
28.571428571428573
38.095238095238095
47.61904761904762
57.142857142857146
66.66666666666667
76.19047619047619
85.71428571428571
95.23809523809524


In [34]:
tech_trade_params

{('AAOI', (2014, 2)): [0.7145961728471981,
  0.6314650667771645,
  0.7145961728471992,
  3.346095689137301],
 ('AAOI', (2015, 6)): [24.430965119664364,
  -0.1439007280974476,
  24.430965119664375,
  1.4413299439521914],
 ('AAOI', (2015, 7)): [28.930402826599575,
  -0.265235986670515,
  28.930402826599597,
  1.5107938779579124],
 ('AAOI', (2016, 12)): [-232.62199864627922,
  5.6840549222142425,
  -232.62199864627922,
  4.834204853125452],
 ('AAOI', (2017, 1)): [-257.5240632848961,
  6.180305104271553,
  -257.5240632848961,
  4.387716720484144],
 ('AAOI', (2017, 7)): [325.5609639001779,
  -4.753173864050503,
  325.5609639001779,
  10.480084470593043],
 ('AAOI', (2017, 8)): [252.44824541984983,
  -3.487042945545287,
  252.4482454198498,
  7.028813129057809],
 ('AAOI', (2017, 9)): [222.0910936840677,
  -3.0008956787479755,
  222.0910936840677,
  5.191660223902015],
 ('AAOI', (2017, 10)): [187.8135783105983,
  -2.4575499041311195,
  187.81357831059825,
  5.278430771649275],
 ('AAOI', (2018,

In [40]:
tech_window_prof = {}
idx = 0

for item in tech_trade_params.items():
    idx+=1
    print(idx*100/len(tech_trade_params.keys()))
    key = item[0]
    params = item[1]
    alpha = params[0]
    beta = params[1]
    mean = params[2]
    sd = params[3]
    s = key[0]
    y_m_tup = key[1] #start of 6 month lookback
    cur_year = y_m_tup[0]
    cur_month = y_m_tup[1]
    
    data = tech_eq_dfs[s]
    gpo = data.groupby(['Year','Month'])
    trading_pd_data = []
    
    for add_month in range(6,9):
        if cur_month + add_month > 12:
            new_month = cur_month + add_month - 12
            new_year = cur_year + 1
        else:
            new_month = cur_month + add_month
            new_year = cur_year
        trading_pd_data.append(gpo.get_group((new_year,new_month)))
    
    trading_data = trading_pd_data[0]
    for i in range(1,len(trading_pd_data)):
        trading_data = pd.concat([trading_data, trading_pd_data[i]], ignore_index=True)
    
    spread = np.subtract(np.array(trading_data['Close']), beta * np.array(trading_data['Close ETF']))
    so = mean + 2*sd
    lo = mean - 2*sd
    lc = mean
    sc = mean
    
    
    prof, ntrades = outsample_pairs_trade(spread,so,lo,sc,lc,10000)
    tech_window_prof[key] = [prof,ntrades]
    

0.00564652738565782
0.01129305477131564
0.01693958215697346
0.02258610954263128
0.028232636928289104
0.03387916431394692
0.039525691699604744
0.04517221908526256
0.050818746470920384
0.05646527385657821
0.062111801242236024
0.06775832862789384
0.07340485601355166
0.07905138339920949
0.08469791078486731
0.09034443817052512
0.09599096555618294
0.10163749294184077
0.10728402032749859
0.11293054771315642
0.11857707509881422
0.12422360248447205
0.12987012987012986
0.13551665725578768
0.1411631846414455
0.14680971202710333
0.15245623941276115
0.15810276679841898
0.1637492941840768
0.16939582156973462
0.17504234895539245
0.18068887634105024
0.18633540372670807
0.1919819311123659
0.1976284584980237
0.20327498588368154
0.20892151326933936
0.21456804065499718
0.220214568040655
0.22586109542631283
0.23150762281197063
0.23715415019762845
0.24280067758328627
0.2484472049689441
0.2540937323546019
0.2597402597402597
0.26538678712591757
0.27103331451157536
0.2766798418972332
0.282326369282891
0.287972

2.5409373235460193
2.546583850931677
2.552230378317335
2.557876905702993
2.5635234330886503
2.569169960474308
2.574816487859966
2.580463015245624
2.5861095426312817
2.5917560700169395
2.5974025974025974
2.6030491247882552
2.608695652173913
2.614342179559571
2.619988706945229
2.6256352343308866
2.6312817617165445
2.6369282891022023
2.6425748164878597
2.6482213438735176
2.6538678712591754
2.6595143986448333
2.665160926030491
2.670807453416149
2.676453980801807
2.6821005081874647
2.6877470355731226
2.6933935629587804
2.6990400903444383
2.704686617730096
2.710333145115754
2.715979672501412
2.7216261998870697
2.727272727272727
2.732919254658385
2.7385657820440428
2.7442123094297006
2.7498588368153585
2.7555053642010163
2.761151891586674
2.766798418972332
2.77244494635799
2.7780914737436477
2.7837380011293056
2.7893845285149634
2.7950310559006213
2.800677583286279
2.8063241106719365
2.8119706380575944
2.8176171654432522
2.82326369282891
2.828910220214568
2.834556747600226
2.8402032749858837


5.166572557876906
5.172219085262563
5.177865612648222
5.183512140033879
5.189158667419537
5.194805194805195
5.200451722190853
5.2060982495765105
5.211744776962168
5.217391304347826
5.223037831733484
5.228684359119142
5.234330886504799
5.239977413890458
5.245623941276115
5.251270468661773
5.256916996047431
5.262563523433089
5.268210050818746
5.273856578204405
5.279503105590062
5.2851496329757195
5.290796160361378
5.296442687747035
5.3020892151326935
5.307735742518351
5.313382269904009
5.319028797289667
5.324675324675325
5.330321852060982
5.335968379446641
5.341614906832298
5.347261434217956
5.352907961603614
5.358554488989272
5.364201016374929
5.369847543760587
5.375494071146245
5.3811405985319025
5.386787125917561
5.392433653303218
5.3980801806888765
5.403726708074534
5.409373235460192
5.41501976284585
5.420666290231508
5.426312817617165
5.431959345002824
5.437605872388481
5.443252399774139
5.448898927159797
5.454545454545454
5.460191981931112
5.46583850931677
5.471485036702428
5.47713

7.8147939017504235
7.820440429136081
7.826086956521739
7.831733483907397
7.837380011293055
7.843026538678712
7.848673066064371
7.854319593450028
7.859966120835686
7.865612648221344
7.871259175607002
7.8769057029926595
7.882552230378318
7.888198757763975
7.893845285149633
7.899491812535291
7.905138339920948
7.910784867306607
7.916431394692264
7.922077922077922
7.92772444946358
7.933370976849238
7.939017504234895
7.944664031620554
7.950310559006211
7.955957086391869
7.961603613777527
7.967250141163185
7.9728966685488425
7.9785431959345
7.984189723320158
7.989836250705816
7.995482778091474
8.001129305477132
8.00677583286279
8.012422360248447
8.018068887634104
8.023715415019764
8.029361942405421
8.035008469791078
8.040654997176736
8.046301524562395
8.051948051948052
8.05759457933371
8.063241106719367
8.068887634105025
8.074534161490684
8.080180688876341
8.085827216261999
8.091473743647656
8.097120271033315
8.102766798418973
8.10841332580463
8.114059853190287
8.119706380575947
8.12535290796

10.581592320722756
10.587238848108413
10.59288537549407
10.59853190287973
10.604178430265387
10.609824957651044
10.615471485036702
10.621118012422361
10.626764539808018
10.632411067193676
10.638057594579333
10.643704121964992
10.64935064935065
10.654997176736307
10.660643704121965
10.666290231507622
10.671936758893281
10.677583286278939
10.683229813664596
10.688876341050253
10.694522868435913
10.70016939582157
10.705815923207227
10.711462450592885
10.717108977978544
10.722755505364201
10.728402032749859
10.734048560135516
10.739695087521174
10.745341614906833
10.75098814229249
10.756634669678148
10.762281197063805
10.767927724449464
10.773574251835122
10.779220779220779
10.784867306606436
10.790513833992096
10.796160361377753
10.80180688876341
10.807453416149068
10.813099943534725
10.818746470920384
10.824392998306042
10.8300395256917
10.835686053077357
10.841332580463016
10.846979107848673
10.85262563523433
10.858272162619988
10.863918690005647
10.869565217391305
10.875211744776962
10

...

87.61151891586674
87.6171654432524
87.62281197063805
87.62845849802372
87.63410502540937
87.63975155279503
87.64539808018068
87.65104460756635
87.656691134952
87.66233766233766
87.66798418972331
87.67363071710898
87.67927724449464
87.68492377188029
87.69057029926596
87.69621682665161
87.70186335403727
87.70750988142292
87.71315640880859
87.71880293619424
87.7244494635799
87.73009599096555
87.73574251835122
87.74138904573687
87.74703557312253
87.75268210050818
87.75832862789385
87.7639751552795
87.76962168266516
87.77526821005083
87.78091473743648
87.78656126482214
87.79220779220779
87.79785431959345
87.8035008469791
87.80914737436477
87.81479390175042
87.82044042913608
87.82608695652173
87.8317334839074
87.83738001129305
87.84302653867871
87.84867306606436
87.85431959345003
87.85996612083568
87.86561264822134
87.87125917560701
87.87690570299266
87.88255223037832
87.88819875776397
87.89384528514964
87.89949181253529
87.90513833992095
87.9107848673066
87.91643139469227
87.92207792207792


90.28797289666855
90.29361942405421
90.29926595143986
90.30491247882553
90.31055900621118
90.31620553359684
90.3218520609825
90.32749858836816
90.33314511575381
90.33879164313947
90.34443817052512
90.35008469791079
90.35573122529644
90.3613777526821
90.36702428006775
90.37267080745342
90.37831733483907
90.38396386222473
90.3896103896104
90.39525691699605
90.40090344438171
90.40654997176736
90.41219649915303
90.41784302653868
90.42348955392434
90.42913608130999
90.43478260869566
90.4404291360813
90.44607566346697
90.45172219085262
90.45736871823829
90.46301524562394
90.4686617730096
90.47430830039525
90.47995482778092
90.48560135516658
90.49124788255223
90.4968944099379
90.50254093732354
90.50818746470921
90.51383399209486
90.51948051948052
90.52512704686617
90.53077357425184
90.53642010163749
90.54206662902315
90.5477131564088
90.55335968379447
90.55900621118012
90.56465273856578
90.57029926595143
90.5759457933371
90.58159232072275
90.58723884810841
90.59288537549408
90.59853190287973


93.00395256916995
93.00959909655562
93.01524562394128
93.02089215132693
93.0265386787126
93.03218520609825
93.03783173348391
93.04347826086956
93.04912478825523
93.05477131564088
93.06041784302654
93.06606437041219
93.07171089779786
93.07735742518351
93.08300395256917
93.08865047995482
93.09429700734049
93.09994353472614
93.1055900621118
93.11123658949747
93.11688311688312
93.12252964426878
93.12817617165443
93.1338226990401
93.13946922642575
93.14511575381141
93.15076228119706
93.15640880858273
93.16205533596838
93.16770186335404
93.17334839073969
93.17899491812535
93.184641445511
93.19028797289667
93.19593450028232
93.20158102766798
93.20722755505365
93.2128740824393
93.21852060982496
93.22416713721061
93.22981366459628
93.23546019198193
93.2411067193676
93.24675324675324
93.25239977413891
93.25804630152456
93.26369282891022
93.26933935629587
93.27498588368154
93.28063241106719
93.28627893845285
93.2919254658385
93.29757199322417
93.30321852060983
93.30886504799548
93.31451157538115


95.65217391304348
95.65782044042913
95.6634669678148
95.66911349520045
95.67476002258611
95.68040654997176
95.68605307735743
95.69169960474308
95.69734613212874
95.7029926595144
95.70863918690006
95.71428571428571
95.71993224167137
95.72557876905702
95.73122529644269
95.73687182382835
95.742518351214
95.74816487859967
95.75381140598532
95.75945793337098
95.76510446075663
95.7707509881423
95.77639751552795
95.78204404291361
95.78769057029926
95.79333709768493
95.79898362507058
95.80463015245624
95.81027667984189
95.81592320722756
95.8215697346132
95.82721626199887
95.83286278938454
95.83850931677019
95.84415584415585
95.8498023715415
95.85544889892716
95.86109542631281
95.86674195369848
95.87238848108413
95.8780350084698
95.88368153585544
95.88932806324111
95.89497459062676
95.90062111801242
95.90626764539807
95.91191417278374
95.91756070016939
95.92320722755505
95.92885375494072
95.93450028232637
95.94014680971203
95.94579333709768
95.95143986448335
95.957086391869
95.96273291925466
95

98.2721626199887
98.27780914737437
98.28345567476002
98.28910220214568
98.29474872953134
98.300395256917
98.30604178430265
98.31168831168831
98.31733483907396
98.32298136645963
98.32862789384528
98.33427442123094
98.3399209486166
98.34556747600226
98.35121400338792
98.35686053077357
98.36250705815924
98.36815358554489
98.37380011293055
98.3794466403162
98.38509316770187
98.39073969508752
98.39638622247318
98.40203274985883
98.4076792772445
98.41332580463015
98.41897233201581
98.42461885940146
98.43026538678713
98.43591191417278
98.44155844155844
98.4472049689441
98.45285149632976
98.45849802371542
98.46414455110107
98.46979107848674
98.47543760587239
98.48108413325805
98.4867306606437
98.49237718802937
98.49802371541502
98.50367024280068
98.50931677018633
98.514963297572
98.52060982495765
98.52625635234331
98.53190287972896
98.53754940711462
98.54319593450028
98.54884246188594
98.5544889892716
98.56013551665725
98.56578204404292
98.57142857142857
98.57707509881423
98.58272162619988
98.

In [41]:
tech_window_prof

{('AAOI', (2014, 2)): [-0.31972390854683397, 1],
 ('AAOI', (2015, 6)): [1.8521628146904554, 1],
 ('AAOI', (2015, 7)): [0.628689679822628, 1],
 ('AAOI', (2016, 12)): [10.438386779422672, 2],
 ('AAOI', (2017, 1)): [9.419446656977641, 2],
 ('AAOI', (2017, 7)): [25.068183110976463, 1],
 ('AAOI', (2017, 8)): [32.71476994205713, 2],
 ('AAOI', (2017, 9)): [9.7462491936312, 2],
 ('AAOI', (2017, 10)): [12.654120689189682, 2],
 ('AAOI', (2018, 12)): [3.093277491048166, 1],
 ('AAOI', (2019, 5)): [2.145311344942389, 1],
 ('AAOI', (2019, 6)): [1.1057521597702689, 2],
 ('AAOI', (2019, 7)): [1.2026844478235308, 2],
 ('AAOI', (2021, 6)): [-2.598724234007406, 1],
 ('AAOI', (2021, 7)): [-1.7922977679091758, 1],
 ('AAOI', (2021, 11)): [0, 0],
 ('AAOI', (2021, 12)): [1.1317626628190567, 1],
 ('AAOI', (2022, 8)): [0, 0],
 ('AAPL', (1999, 8)): [0.06895643752086178, 1],
 ('AAPL', (1999, 9)): [0.1172495749214113, 1],
 ('AAPL', (1999, 10)): [0.16869793316822013, 2],
 ('AAPL', (1999, 11)): [0.20732771506316783,

In [42]:
sum_ = 0
for item in tech_window_prof.items():
    sum_+=item[1][0]
print(sum_)

-3625961925.0203266


# Healthcare

In [35]:
hc_trade_params = {} #maps (tech equities, start of window year-month tuple) tuple to values that need to be known to pairs trade [alpha, beta, mean, sd]
hc_equities = hc_eq_dfs.keys()
idx = 0
for s in hc_equities:
    idx+=1
    if idx % 50 == 0:
        print(idx*100/len(hc_equities))
    df = hc_eq_dfs[s]
    gbo = df.groupby(['Year','Month'])
    gb_keys = []
    for key, frame in gbo:
        gb_keys.append(key)
    for y_m_key in gb_keys[:-8]:
        lookback_df_list = [] 
        first_year = y_m_key[0]
        first_month = y_m_key[1]
        for i in range(6):
            cur_month = first_month + i
            if cur_month < 13:
                cur_year = first_year
            else:
                cur_month-=12
                cur_year = first_year+1
            cur_gp = gbo.get_group((cur_year,cur_month))
            lookback_df_list.append(cur_gp)
        lookback_df = lookback_df_list[0]
        for i in range(1,len(lookback_df_list)):
            lookback_df = pd.concat([lookback_df,lookback_df_list[i]],ignore_index=True)
        try:
            ols = sm.OLS(np.array(lookback_df['Close']),sm.add_constant(np.array(lookback_df['Close ETF']).reshape((-1,1))))
            res = ols.fit()
            alpha = res.params[0] #y-int
            beta = res.params[1]
            residuals = np.subtract(np.array(lookback_df['Close']),ols.predict(res.params)) + np.ones(lookback_df.shape[0])*res.params[0]
            cur_pval = adfuller(residuals)[1]
            corr_coef = np.corrcoef(np.hstack((np.array(lookback_df['Close']).reshape(-1),np.array(lookback_df['Close ETF']).reshape(-1))))
            if cur_pval <= pval_max and corr_coef >= corr_thresh:
                #can trade spread
                mean = np.mean(residuals)
                sd = np.std(residuals)
                hc_trade_params[(s,(y_m_key))] = [alpha,beta,mean,sd]
        except:
            continue
        

5.302226935312832
10.604453870625663
15.906680805938494
21.208907741251327
26.511134676564158
31.81336161187699
37.11558854718982
42.417815482502654
47.720042417815485
53.022269353128316
58.324496288441146
63.62672322375398
68.9289501590668
74.23117709437965
79.53340402969248
84.83563096500531
90.13785790031814
95.44008483563097


# Energy

# Financials

# Consumer Staples

# Utilities

# Consumer Discretionary

# Basic Materials

# Industrials

# Telecommunications 

# Real Estate